# Naive RAG

In [5]:
import os
import json
import numpy as np
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer

# 1. 환경변수 로드
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# 2. 임베딩된 청크 JSON 로드
with open("output_chunks_with_embeddings.json", "r", encoding="utf-8") as f:
    chunk_data = json.load(f)

# 3. Document + embedding 분리
documents = []
embeddings = []

for item in chunk_data:
    documents.append(Document(
        page_content=item["text"],
        metadata={"filename": item["filename"], "chunk_index": item["chunk_index"]}
    ))
    embeddings.append(np.array(item["embedding"]))

# 4. Naive VectorStore
class NaiveVectorStore:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = np.array(embeddings)

    def similarity_search_by_vector(self, query_vector, k=3):
        query_norm = query_vector / np.linalg.norm(query_vector)
        doc_norms = self.embeddings / np.linalg.norm(self.embeddings, axis=1, keepdims=True)
        sims = np.dot(doc_norms, query_norm)
        top_k_idx = sims.argsort()[::-1][:k]
        return [self.documents[i] for i in top_k_idx]

# 5. 질문 임베딩용 SentenceTransformer 모델 로드
sbert_model = SentenceTransformer("intfloat/multilingual-e5-large-instruct")  # 청크와 동일한 모델일 것

# 6. VectorStore 인스턴스화
naive_store = NaiveVectorStore(documents, embeddings)

# 7. Retriever: 질문 → 유사 문서 text 추출
retriever = RunnableLambda(lambda q: naive_store.similarity_search_by_vector(
    sbert_model.encode("query: " + q), k=3)
)

# 8. 프롬프트 템플릿
prompt = ChatPromptTemplate.from_template(
    """다음은 법률 문서에서 발췌한 내용입니다:\n\n{context}\n\n
    위 내용을 바탕으로 질문에 정확히 답변하세요.
    반드시 다음 사항을 지켜주세요:
    - 답변은 법률적 근거가 되는 문장 또는 조문을 인용해 설명하세요.
    - 어떤 법에서 발췌된 내용인지 언급하세요. (예: 종합부동산세법 제1조)
    - 만약 문서에 명확한 답이 없다면 "문서에서 해당 정보를 찾을 수 없습니다"라고 말하세요.

    질문: {question}
    답변:"""
)

# 9. GPT LLM 정의
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# 10. LangChain Expression Language 체인
chain = (
    {"docs": retriever, "question": RunnablePassthrough()}
    | RunnableLambda(lambda x: {
        "context": "\n\n".join(doc.page_content for doc in x["docs"]),
        "question": x["question"]
    })
    | prompt
    | llm
    | RunnableLambda(lambda msg: msg.content)
)

# 11. 질의 실행
question = "종합부동산세법의 목적을 법령 조문을 근거로 하여 설명해주세요."
response = chain.invoke(question)
print("답변:", response)

답변: 종합부동산세법의 목적은 국가재정 수요의 충당을 넘어서 부동산 가격 안정 등의 적극적인 목적을 추구하는 유도적 형성적 기능을 지닌 정책적 조세입니다. 이는 당해 조세가 추구하는 특별한 정책 목적과의 관계에서 그 수단인 조세의 부과가 정책목적 달성에 적합하고 필요한 한도 내에 그쳐야 할 뿐만 아니라 그 정책 목적에 의하여 보호하고자 하는 공익과 침해되는 사익 사이에도 비례관계를 유지하여 과잉금지의 원칙에 어긋나지 않아야 한다고 명시되어 있습니다. 또한, 이 법은 주택 수에 따라 세율 등의 적용을 구분하고, 주택 이상 또는 조정대상지역 내 주택 소유자에 대해 세율체계를 달리 정하는 방식으로 변경되었습니다. 이는 부동산 자산 총액 대비 보유세 비중이 평균의 절반 수준에 불과하고, 재산과세에서 보유세 비중이 낮은 수준에 머물고 있음을 반영하여 과세형평성을 제고하고, 대체적으로 세율을 인상하면서 부의 편중현상을 완화하며 경제적 효율성을 높이기 위한 것입니다. 이는 법제처 개정이유에서 확인할 수 있습니다.


## Cursor AI 제안 버전

## RAG_with_Retriever 사용법

`RAG_with_Retriever.py`를 사용하려면 `ExperimentConfig` 객체를 생성해서 전달해야 합니다.

### 지원하는 리트리버 종류:
1. **naive_cosine_e5**: 임베딩 기반 코사인 유사도 검색 (기본값)
2. **bm25**: 키워드 기반 BM25 검색
3. **tfidf**: TF-IDF 벡터 기반 검색
4. **hybrid_rrf**: Dense + Sparse 하이브리드 검색 (Reciprocal Rank Fusion)

### 주요 파라미터:
- `k_ctx`: 생성에 사용할 문서 수 (기본값: 5)
- `k_in`: 리랭커용 후보군 수 (기본값: 50)
- `k_dbg`: 디버깅용 출력 수 (기본값: 10)


In [1]:
from Improved_Basic_RAG import LegalRAGSystem

# RAG 시스템 초기화
rag=LegalRAGSystem()

# 질문하기
answer=rag.ask_question("종합부동산세는 무엇인가요?")

# 유사도 검색 테스트
docs=rag.test_similarity_search("납세의무자")

✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\Naive_RAG\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 1057)
⚙️ RAG 컴포넌트 설정 중...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!

🤖 질문 처리 중: 종합부동산세는 무엇인가요?
--------------------------------------------------
📚 **참고한 문서:**
  1. 판례_71063.txt_chunk_4
  2. 판례_178579.txt_chunk_3
  3. 판례_237299.txt_chunk_3

💡 **답변:**
종합부동산세는 주택 및 토지에 대해 부과되는 세금으로, 납세의무자가 소유한 부동산의 공시가격을 기준으로 과세됩니다. 이 세금은 주택에 대한 종합부동산세와 토지에 대한 종합부동산세로 나뉘며, 주택에 대한 종합부동산세는 주택의 공시가격을 합산한 금액에서 일정 금액을 공제한 후, 정해진 세율을 적용하여 계산됩니다.

구체적으로, 주택에 대한 종합부동산세의 과세표준은 다음과 같이 정의됩니다:

1. **과세표준**: 납세의무자별로 주택의 공시가격을 합산한 금액에서 6억 원을 공제한 금액으로 하며, 이 금액이 영보다 작을 경우에는 영으로 간주합니다(제8조 ①항).
2. **세율**: 과세표준에 따라 차등 세율이 적용되며, 3억 원 이하, 3억 원 초과 14억 원 이하, 14억 원 초과 94억 

In [1]:
from Improved_Basic_RAG import LegalRAGSystem

# RAG 시스템 초기화
rag=LegalRAGSystem()

# 질문하기
answer=rag.ask_question("종합부동산세의 과세 기준은 어떻게 돼?")

# 유사도 검색 테스트
docs=rag.test_similarity_search("부동산세")

✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\PDF 파일 기반 Naive RAG\output_chunks_with_embeddings.json
✅ 6569개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 종합부동산세법 현시행 제 호 일부개정 [ 2025.03.14.] [ 20779 , 2025.03.14.] 제조 목적 1 이 법은 고액의 부동산 보유자에 대하여 종합부동산세를 부과하...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 6569)
⚙️ RAG 컴포넌트 설정 중...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!

🤖 질문 처리 중: 종합부동산세의 과세 기준은 어떻게 돼?
--------------------------------------------------
📚 **참고한 문서:**
  1. 서면인터넷방문상담5팀-241.pdf_chunk_2
  2. 서면인터넷방문상담5팀-0034.pdf_chunk_4
  3. 서면인터넷방문상담5팀-3226.pdf_chunk_1

💡 **답변:**
종합부동산세의 과세 기준은 다음과 같습니다.

1. **과세기준일**: 종합부동산세의 과세기준일은 지방세법 제190조에 규정된 재산세의 과세기준일로 정해져 있습니다. 이는 매년 특정일에 재산의 가치를 기준으로 세금이 부과된다는 의미입니다.

2. **과세방법**: 종합부동산세는 두 가지 방식으로 과세됩니다:
   - **종합합산과세대상**: 국내에 소재하는 토지의 공시가격을 합산한 금액이 억원을 초과하는 경우, 개인의 경우 세대별로 합산한 금액이 해당됩니다. 이 경우 납세의무자는 종합부동산세를 납부할 의무가 있습니다.
   - **별도합산과세대상**: 국내에 소재하는 토지의 공시가격을 합산한 금액이 억원이하인 경우, 이 역시 개인의 

In [2]:
from RAG_with_Retriever import LegalRAGSystem, ExperimentConfig

In [4]:
# RAG_with_Retriever를 사용하는 올바른 방법
from RAG_with_Retriever import LegalRAGSystem, ExperimentConfig

# 기본 설정으로 ExperimentConfig 생성
config = ExperimentConfig(
    embeddings_file="output_chunks_with_embeddings.json",
    retriever_id="naive_cosine_e5",  # 리트리버 종류: "naive_cosine_e5", "bm25", "tfidf", "hybrid_rrf"
    k_ctx=6,  # 생성에 사용할 문서 수
    k_in=50,  # 리랭커용 후보군 수  
    k_dbg=10  # 디버깅용 출력 수
)

# RAG 시스템 초기화
rag = LegalRAGSystem(config)

# 질문하기
answer = rag.ask_question("종합부동산세는 무엇인가요?")

# 유사도 검색 테스트
docs = rag.test_similarity_search("납세의무자")


✅ 환경/설정 준비 완료, config 저장: exp_outputs\retriever_baseline_config.json
🚀 시스템 초기화 시작
📂 임베딩 데이터 로드: output_chunks_with_embeddings.json
✅ 문서 청크: 1057개
📄 첫 청크: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 VectorStore 생성 (type: naive_cosine_e5)...
✅ VectorStore 준비(문서 수: 1057)
🔧 Retriever 설정 중... (type: naive_cosine_e5)
✅ Retriever 준비 완료: naive_cosine_e5
⚙️ RAG 컴포넌트 구성...
✅ RAG 체인 구성 완료
🎉 초기화 완료

🤖 질문: 종합부동산세는 무엇인가요?
📚 참고 문서:
  1. 판례_71063.txt_chunk_4
  2. 판례_178579.txt_chunk_3
  3. 판례_237299.txt_chunk_3
  4. 판례_237251.txt_chunk_6
  5. 판례_180048.txt_chunk_8
  6. 판례_193173.txt_chunk_4

💡 답변:
종합부동산세는 부동산을 보유하고 있는 납세의무자에게 부과되는 세금으로, 주택 및 토지에 대한 과세를 포함합니다. 이 세금은 특정 기준을 초과하는 부동산의 공시가격에 대해 부과되며, 주택의 경우에는 납세의무자별로 주택의 공시가격을 합산한 금액에서 6억 원을 공제한 금액을 과세표준으로 삼습니다(제8조 ①항). 

종합부동산세의 세율은 과세표준에 따라 다르게 적용되며, 주택의 경우 다음과 같은 세율이 적용됩니다(제9조 ①항):
- 3억 원 이하: 1천분의 10
- 3억 원 초과 14억 원 이하: 1천분의 15
- 14억 원 초과 94억 원 이하: 1천분의 20
- 94억 원 초과: 1천분의 

In [3]:
# 다른 리트리버 테스트 예시 - BM25
config_bm25 = ExperimentConfig(
    embeddings_file="output_chunks_with_embeddings.json",
    retriever_id="bm25",  # BM25 키워드 기반 검색
    k_ctx=5,
    k_in=50,
    k_dbg=10
)

# BM25 리트리버로 RAG 시스템 초기화
rag_bm25 = LegalRAGSystem(config_bm25)

# 동일한 질문으로 테스트
print("=== BM25 Retriever 결과 ===")
answer = rag_bm25.ask_question("종합부동산세는 무엇인가요?")


✅ 환경/설정 준비 완료, config 저장: exp_outputs\retriever_baseline_config.json
🚀 시스템 초기화 시작
📂 임베딩 데이터 로드: output_chunks_with_embeddings.json
✅ 문서 청크: 1057개
📄 첫 청크: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 VectorStore 생성 (type: bm25)...
🔧 Retriever 설정 중... (type: bm25)
✅ Retriever 준비 완료: bm25
⚙️ RAG 컴포넌트 구성...
✅ RAG 체인 구성 완료
🎉 초기화 완료
=== BM25 Retriever 결과 ===

🤖 질문: 종합부동산세는 무엇인가요?
📚 참고 문서:
  1. 판례_70802.txt_chunk_5
  2. 판례_70802.txt_chunk_4
  3. 판례_70802.txt_chunk_15
  4. 판례_199732.txt_chunk_5
  5. 판례_241679.txt_chunk_10

💡 답변:
종합부동산세는 2005년 1월 5일에 제정된 법률 제7328호에 따라 도입된 세금으로, 부동산을 보유하는 자에게 부과되는 국세입니다. 이 세금은 특정 기준금액을 초과하는 부동산에 대해 높은 세율로 부과되며, 그 목적은 부동산 보유에 대한 조세 부담의 형평을 제고하고 부동산 가격 안정을 도모하는 것입니다.

종합부동산세는 지방세인 재산세와 동일한 과세 객체에 대해 중과세하는 형태로, 재산세로 부과된 세액을 종합부동산세에서 공제하는 장치가 마련되어 있습니다(구 종합부동산세법 제9조 제3항, 제4항). 또한, 1주택자에 대해서는 5억 원을 추가로 공제하며(같은 법 제8조 제1항), 연령 및 장기 보유 여부에 따라 세액을 공제하는 규정도 존재합니다(같은 법 제9조 제5항, 제6항, 제7항).

종합부동산세는 